In [30]:
import cv2
from PIL import Image
from skimage.util import img_as_ubyte
import numpy as np
def block_no_expand(name):
    im=np.array(name)
    im_lab=cv2.cvtColor(im,cv2.COLOR_RGB2HSV)
    im_lab[:,:,-1]=np.interp(im_lab[:,:,-1],(0,255),(0,255/4))
    im_rgb=cv2.cvtColor(im_lab,cv2.COLOR_HSV2RGB)
    im_ht=Image.fromarray(im_rgb).convert("1")
    im_ht=img_as_ubyte(im_ht)
    # gray_level(im_ht,(2,4),True)
    return im_ht

In [39]:
import os
from tqdm import tqdm
for ds in os.listdir('.'):
    if ds.startswith('dataset'):
        for i in tqdm(os.listdir(ds)):
            for j in os.listdir(os.path.join(ds,i)):
                path = os.path.join(ds,i)
                name= j.split('.')[0]
                if j.endswith('bmp'):
                    bmp = Image.open(f"{path}/{name}.bmp")
                    bmp.save(f"{path}/{name}.png", "PNG")

100%|██████████| 10/10 [00:01<00:00,  6.86it/s]


In [7]:
import os
from tqdm import tqdm
dataset = './dataset_ori'
dataset_NoEx = './dataset_NoEx'
dataset_re = './dataset_re'
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_NoEx,i,j)
        Image.fromarray(block_no_expand(Image.open(path).convert("RGB"))).save(path_save)

100%|██████████| 10/10 [00:00<00:00, 10.60it/s]


In [13]:
from skimage.metrics import peak_signal_noise_ratio


def re_halftone(pre, path, path_ori):
    # Load image
    #orig=path
    
    img=img_as_ubyte(Image.open(path))
    pil_halftone = Image.open(path).convert('L').convert("1")
    halftone = img_as_ubyte(pil_halftone)
    ref = img_as_ubyte(Image.open(path_ori).convert('L').convert("1"))
    def IHT(radius=3,sigma_s=100,cnts=3):
        dehalftoned=halftone.copy()
        for i in range(cnts):
            dehalftoned = cv2.bilateralFilter(I,radius,sigma_s,sigma_s,dst=dehalftoned)
        return dehalftoned

    best_psnr = 0
    best_image = None
    best_ehist = None
    for radius in range(1, 15,2): # 超参数
        for sigma_s in [10,20,30,50,100,150,200,400,500,600]: # 超参数
            for cnts in range(1,9): # 超参数
                I=cv2.GaussianBlur(halftone.copy(),(radius,radius),0)
                dehalftoned=IHT(radius,sigma_s,cnts)
                ehist=cv2.equalizeHist(dehalftoned) # ehist 为增强解密图像
                name=f"{pre}/{cnts}_{radius}_{sigma_s}.png"
                psnr = peak_signal_noise_ratio(ehist,ref)
                if psnr > best_psnr:
                    best_psnr = psnr
                    best_image = name
                    best_ehist = ehist

    print("Best PSNR:", best_psnr)
    print("Best image:", best_image)
    return best_ehist

In [ ]:
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset_NoEx,i,j)
        path_ori = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_re,i,j)
        Image.fromarray(re_halftone(i,path, path_ori)).save(path_save)

In [15]:

from PIL import Image
import numpy as np
def halftone(img_name, convert_to_byte=True):
    img = Image.open(img_name)
    img = img.resize([i//2*2 for i in img.size])
    img_arr = np.array(img.convert("1"))
    if convert_to_byte:
        return img_as_ubyte(img_arr)
    return img_arr

In [17]:
def basic_single_vcs(path="lena.png", to_save=None):
    img_arr = halftone(path)
    share1 = np.random.choice([True, False], img_arr.shape)
    share2 = np.zeros_like(share1)
    for (i, j), p in np.ndenumerate(img_arr):
        if p:
            share2[i, j] = share1[i, j]
        else:
            share2[i, j] = not share1[i, j]
    if to_save:
        Image.fromarray(share1 & share2).save(to_save)
    else:
        return share1, share2

In [21]:
dataset_p3 = './dataset_p3'
import os
from tqdm import tqdm
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_p3,i,j)
        basic_single_vcs(path,path_save)

100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


In [22]:
def basic_vcs2(img_name="lena.png", to_save=None):
    img_arr = halftone(img_name)
    share1 = np.zeros((img_arr.shape[0],2*img_arr.shape[1]),bool)
    share1[:][::2]=True
    share2 = np.zeros_like(share1)
    for (i, j), p in np.ndenumerate(img_arr):
            if p:
                share2[i, 2*j:2*(j+1)] = share1[i, 2*j:2*(j+1)]
            else:
                share2[i, 2*j:2*(j+1)] = np.invert(share1[i, 2*j:2*(j+1)])
    if to_save:
        Image.fromarray(share1 & share2).save(to_save)
    else:
        return share1, share2

In [23]:
dataset_p1 = './dataset_p1'
import os
from tqdm import tqdm
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_p1,i,j)
        basic_vcs2(path,path_save)

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]


In [24]:
def basic_rg_vcs(img_name="lena.png", to_save=None, n=8):
    img_arr = np.invert(Image.open(img_name).convert("1"))
    shares = np.zeros((n,)+img_arr.shape, bool)
    for (i, j), p in np.ndenumerate(img_arr):
        if p:
            for ind in range(n):
                shares[ind, i, j] = np.random.choice([True, False])
        else:
            v = np.random.choice([True, False])
            shares[:, i, j] = v
    if to_save:
        Image.fromarray(shares[0] & shares[1] & shares[2] & shares[3] & shares[4] & shares[5] & shares[6] & shares[7]).save(to_save)
    else:
        return shares

In [26]:
dataset_p2 = './dataset_p2'
import os
from tqdm import tqdm
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_p2,i,j)
        basic_rg_vcs(path,path_save)

100%|██████████| 10/10 [06:12<00:00, 37.23s/it]


In [27]:
def basic_prob_vcs(img_name="lena.png", to_save=None):
    B0 = np.array([[1, 0], [1, 0]], bool)
    B1 = np.array([[0, 1], [1, 0]], bool)
    img_arr = halftone(img_name)
    share1 = np.zeros_like(img_arr, bool)
    share2 = np.zeros_like(img_arr, bool)
    for (i, j), p in np.ndenumerate(img_arr):
        if p:
            (share1[i, j], share2[i, j]) = B0[:, np.random.choice(B0.shape[1])]
        else:
            (share1[i, j], share2[i, j]) = B1[:, np.random.choice(B0.shape[1])]
    if to_save:
        Image.fromarray(share1 & share2).save(to_save)
    else:
        return share1, share2

In [28]:
dataset_p4 = './dataset_p4'
import os
from tqdm import tqdm
for i in tqdm(os.listdir(dataset)):
    for j in os.listdir(os.path.join(dataset,i)):
        path = os.path.join(dataset,i,j)
        path_save = os.path.join(dataset_p4,i,j)
        basic_prob_vcs(path,path_save)

100%|██████████| 10/10 [01:58<00:00, 11.86s/it]


In [29]:
with open ('pairs.txt','w') as f:
    for i in range(300):
        dir = np.random.choice(os.listdir(dataset))
        img1 = np.random.choice(os.listdir(os.path.join(dataset,dir))).split('.')[0].split('_')[1]
        img2 = np.random.choice(os.listdir(os.path.join(dataset,dir))).split('.')[0].split('_')[1]
        while img1 == img2:
            img2 = np.random.choice(os.listdir(os.path.join(dataset,dir))).split('.')[0].split('_')[1]
        f.write(f"{dir} {img1} {img2} \n")
    
    for i in range(300):
        dir1 = np.random.choice(os.listdir(dataset))
        dir2 = np.random.choice(os.listdir(dataset))
        while dir1 == dir2:
            dir2 = np.random.choice(os.listdir(dataset))
        img1 = np.random.choice(os.listdir(os.path.join(dataset,dir1))).split('.')[0].split('_')[1]
        img2 = np.random.choice(os.listdir(os.path.join(dataset,dir2))).split('.')[0].split('_')[1]
        f.write(f"{dir1} {img1} {dir2} {img2} \n")